In [80]:
from numpy import *
import time
import sys
import scipy.io

print("enter file name: ", end="")
fil = input()  

fil= fil +".mat"
print("Reading data from file '" + fil + "'")

if fil==None:
    print('File '+repr(fil)+' does n ot exist')
    exit()

print('Datafile: '+repr(fil))
mat=scipy.io.loadmat(fil,squeeze_me=True)

c=mat['c']
A=mat['A']
b=mat['b']
bix=asarray(mat['bix'],dtype=int)
zcheat=mat['zcheat']
if 'xcheat' in mat:
    xcheat=mat['xcheat']
else:
    xcheat=0

bix=bix-1
[m,n] = shape(A)
print('Rows: '+repr(m)+' cols: '+repr(n))

print('c:',c)
print('b:',b)
print('A:',A)
print('bix:',bix)
print('xcheat:',xcheat)
print('zcheat:',zcheat)


enter file name: Reading data from file 'lab1-test.mat'
Datafile: 'lab1-test.mat'
Rows: 2 cols: 4
c: [ 1. -3.  0.  0.]
b: [6. 5.]
A: [[-1.  2.  1.  0.]
 [ 1.  1.  0.  1.]]
bix: [2 3]
xcheat: [1.33333333 3.66666667 0.         0.        ]
zcheat: -9.666666666666668


In [81]:
from ctypes import sizeof
import numpy as np
import time
import sys

rc_min = np.nanmin(c)


t1=time.time()

[m,n] = np.shape(A)
print( 'Rows: '+repr(m)+' cols: '+repr(n))

# Create nix
nix=np.setdiff1d(range(n), bix) 

B  = A[:, bix]
N  = A[:, nix]
cB = c[bix]
cN = c[nix]
print(B)
# Start iterations
iter = 0
while iter >= 0:
    iter+=1            


    inverseB = np.linalg.inv(B)
    bHatt = np.matmul(inverseB, b)
    
    y = np.transpose(np.matmul(np.transpose(cB), inverseB))

    z= np.matmul(np.transpose(b),y)
    cNhatt = (cN - np.matmul(np.transpose(N),y))

    inkvarPlats = np.argmin(cNhatt)
    # inkvar = int(np.where(cNhatt==np.nanmin(cNhatt))[0][0]) #Tidigare cKhatt min
    rc_min = cNhatt[inkvarPlats]
    inkvar = nix[inkvarPlats]

  
    # calc right-hand-sides and reduced costs
    
     
    # --------

    # calc most negative reduced cost, rc_min,
    # and index for entering variable, inkvar
    # --------


    if rc_min >= -1.0E-12:
        print ('Ready')
        iter=-1
 
        # construct solution, x, and check it
        # --------
       
        x = np.full([np.size(c)], 0.00)
        print("bhatt: ", bHatt)
        

        p = bix.argsort()
        # x =  np.add(x, )
        t = bHatt[p]
        x[:len(t)] += t

        
        z = np.matmul(np.transpose(b),y)
        print("funktionsvärde: ",z)

        diffx = np.linalg.norm(x-xcheat)
        diffz = z-zcheat
        print ('xdiff: '+repr(diffx))
        print ('zdiff: '+repr(diffz))
    else:
        # calc entering column, a
        # -------- 
        ak = np.matmul(inverseB, A[:, inkvar])
        print("ak: ",ak)
        if max(ak) <= 0 :
            # unbounded solution
            print('Unbounded solution!')
            iter=-1
        else:
            # calc leaving var, utgvar
            # --------
            # alkHatt= np.divide(b,np.transpose(N)[inkvarPlats]) #inkommande var kol genom b (bhat), nu bestäms utgående av det minsta
            alkHatt = bHatt/ak

            utgvarPlats = np.argmin(alkHatt)
            utgvar = bix[utgvarPlats]
    


            print(' Iter: '+repr(iter)+' z: '+repr(z)+' rc: '+repr(rc_min)+' ink: '+repr(inkvar+1)+' utg: '+repr(utgvar+1))

            # make new partition
            # -------
            
           
            bix[utgvarPlats] = nix[inkvarPlats]


            B = A[:,bix]
            N = A[:,nix]
            cB = c[bix]
            cN = c[nix]
            

   



elapsed = time.time() - t1
print('Elapsed time: '+repr(elapsed))

Rows: 2 cols: 4
[[1. 0.]
 [0. 1.]]
ak:  [2. 1.]
 Iter: 1 z: 0.0 rc: -3.0 ink: 2 utg: 3
ak:  [-0.5  1.5]
 Iter: 2 z: -9.0 rc: -0.5 ink: 1 utg: 2
ak:  [-2.  3.]
 Iter: 3 z: -6.0 rc: -1.0 ink: 2 utg: 1
ak:  [-0.5  1.5]
 Iter: 4 z: -9.0 rc: -0.5 ink: 1 utg: 2
ak:  [-2.  3.]
 Iter: 5 z: -6.0 rc: -1.0 ink: 2 utg: 1
ak:  [-0.5  1.5]
 Iter: 6 z: -9.0 rc: -0.5 ink: 1 utg: 2
ak:  [-2.  3.]
 Iter: 7 z: -6.0 rc: -1.0 ink: 2 utg: 1
ak:  [-0.5  1.5]
 Iter: 8 z: -9.0 rc: -0.5 ink: 1 utg: 2
ak:  [-2.  3.]
 Iter: 9 z: -6.0 rc: -1.0 ink: 2 utg: 1
ak:  [-0.5  1.5]
 Iter: 10 z: -9.0 rc: -0.5 ink: 1 utg: 2
ak:  [-2.  3.]
 Iter: 11 z: -6.0 rc: -1.0 ink: 2 utg: 1
ak:  [-0.5  1.5]
 Iter: 12 z: -9.0 rc: -0.5 ink: 1 utg: 2
ak:  [-2.  3.]
 Iter: 13 z: -6.0 rc: -1.0 ink: 2 utg: 1
ak:  [-0.5  1.5]
 Iter: 14 z: -9.0 rc: -0.5 ink: 1 utg: 2
ak:  [-2.  3.]
 Iter: 15 z: -6.0 rc: -1.0 ink: 2 utg: 1
ak:  [-0.5  1.5]
 Iter: 16 z: -9.0 rc: -0.5 ink: 1 utg: 2
ak:  [-2.  3.]
 Iter: 17 z: -6.0 rc: -1.0 ink: 2 utg: 1
ak:  [-0.

KeyboardInterrupt: 